In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Approximating functions for cosmology ##

If you aren't familiar with standard methods of function approximation, look at the "TableConstruction" iPython notebook.  That material is something graduate students in technical fields are supposed to "just know" (somehow).  We will use some of the tricks below.

### Massive neutrinos ###

Let us begin by considering the calculation of "background" cosmological quantities (i.e. distances, ages, ...) in cosmologies including massive neutrinos.  All such quantities essentially reduce to integrals over the (inverse of the) Hubble parameter, $H(z)$.  To be concrete let us consider the computation of the comoving distance, $\chi$, in a spatially flat cosmology:
$$
  \frac{d\chi}{dz} = \frac{c}{H(z)} \quad\Rightarrow\quad
  \chi(z) = \frac{c}{H_0}\int_0^z\frac{dz'}{E(z')}
$$
where $E(z)=H(z)/H_0$.  Using the Friedmann equation
$$
  E(z) = \sqrt{ \Omega_m (1+z)^3 + \Omega_r (1+z)^4 + \cdots + \Omega_\Lambda }
$$
where the $\cdots$ stand in for the neutrino energy density.  If neutrinos were always massive (say below $z\sim 10$ in our Universe) then they could be lumped in with the matter term.  If they were always relativistic (say at $z\sim 10^3$) they could be lumped in with the radiation.  Let us start with low $z$ just as a warm up.

In [ ]:
def chi(z,OmM=0.3):
    """A fairly dumb routine to compute the comoving distance to redshift z.
       This is not how you'd actually code it -- but since astropy or various
       Boltzmann codes have already "coded" it we can always use that when we
       need to and just doing something dumb here!"""
    Lhub=2997.925 # c/H0 in Mpc/h.
    zp = np.linspace(0,z,5000)
    Ez = np.sqrt( OmM*(1+zp)**3 + (1-OmM) )
    cz = np.trapz(1.0/Ez,x=zp) * Lhub
    return(cz)
    #
# Do some sanity checks -- should be about 2300Mpc/h.
print(chi(1.0))

It's not hard to add in some radiation, going as $(1+z)^4$, but what about the neutrinos?

What if we want to find e.g. the distance to last scattering and so have to integrate **through** the time when they go from relativistic to non-relativistic? Then we need to be able to include the neutrino energy density.

### Neutrino energy density ###

Neutrinos fall out of thermal equilibrium while they are still relativistic, at which point they have a Fermi-Dirac distribution (with $\mu=0$ assumed).  After that point their momentum just shifts as $p\propto (1+z)$ so they keep a F-D distribution of momenta but with a temperature falling as $T\propto (1+z)$.  Thus the sum of the neutrino and photon energy densities is
\begin{equation}
  \rho_{\gamma+\nu} = \frac{\pi^2}{15} \left[ T_{\rm cmb}^4 +
  T_\nu^4 \sum_i\mathcal{J}\left(\frac{m_ic^2}{k_BT_\nu}\right)\right]
\end{equation}
for neutrinos of mass $m_i$ where $T_\nu=(4/11)^{1/3}\,T_{\rm cmb}$, $T_{\rm cmb}\propto (1+z)$, and
\begin{equation}
  \mathcal{J}(r) = \frac{15}{\pi^4}\int_0^\infty x^2\,dx
  \ \frac{\sqrt{x^2+r^2}}{e^x+1}
\end{equation}
Straightforwardly $\mathcal{J}(0)=7/8$, $\mathcal{J}'(0)=0$:
Setting $r=0$ gives the standard integral:
\begin{equation}
  I_1(4) = \int_0^\infty\frac{x^3\,dx}{e^x+1}
         = \left[1-\left(\frac{1}{2}\right)^3\right]\ \Gamma(4)\zeta(4)
         = \frac{7}{8} \Gamma(4)\frac{2^3\pi^4B_2}{4!}
         = \frac{7}{8} \frac{\pi^4}{15}
\end{equation}
so $\mathcal{J}(0)=7/8$.  The fact that $\mathcal{J}'(0)=0$ follows immediately from the $r^2$ dependence in the $\sqrt{x^2+r^2}$.  Also $\rho_\nu\propto a^{-3}$ when $r\gg 1$.
A Pade approximant to $\mathcal{J}(r)$ can be written
\begin{equation}
  \mathcal{J}(r)\simeq \frac{a_0+a_1r+a_2r^2}{1+b_1r}
\end{equation}
The expansion in powers of $1/r$ follows from
\begin{equation}
  \frac{\mathcal{J}}{r} = \int_0^\infty\frac{x^2\,dx\ \sqrt{1+(x/r)^2}}{e^x+1}
  = \int_0^\infty\frac{x^2\,dx\ \left(1+(1/2)(x/r)^2-(1/8)(x/r)^4+\cdots\right)}
                      {e^x+1}
\end{equation}
and using standard integrals:
\begin{equation}
  \frac{\mathcal{J}}{r} \simeq \frac{45\zeta(3)}{4\pi^4} +
  \frac{675\zeta(5)}{4\pi^4}\,\frac{1}{r^2} -
  \frac{42525\zeta(7)}{32\pi^4}\,\frac{1}{r^4} + \cdots
  \simeq 0.277675 + \frac{1.79363}{r^2} - \frac{13.7564}{r^4} + \cdots
\end{equation}
Since $\mathcal{J}/r\to$const as $r\to\infty$ we have
$\rho\propto T^4r\propto T^3m\propto a^{-3}$.
Finding the coefficients $a_i$ and $b_i$ is a simple matter of solving the simultaneous equations
\begin{equation}
  a_0=\frac{7}{8} \quad , \quad
  a_1-a_0b_1 = 0 \quad , \quad
  a_2 = 0.277675\,b_1 \quad , \quad
  a_1b_1-a_2 = 0
\end{equation}
The solution of interest gives
\begin{equation}
  \mathcal{J}(r) \approx \frac{0.875+0.277675\,r+0.0881182\,r^2}
                              {1+0.317343\,r}
\end{equation}
which matches $\mathcal{J}(r)$ to better than $3\%$ for all $r$, being worst for $r$ of a few.

This may be good enough, given that neutrinos are a minority component when they become non-relativistic, but it is not too difficult to go to higher order if necessary.

In [ ]:
# Let's see how accurate this approximation is.
# The exact integral.
def exactJ(r):
    """Does the integral for J(r) numerically."""
    xx  = np.linspace(0,10,5000)
    res = np.trapz( xx**2*np.sqrt(xx**2+r**2)/(np.exp(xx)+1),x=xx )
    return( 15/np.pi**4 * res )
# The Pade approximant
num=np.poly1d([0.0881182,0.277675,0.875])
den=np.poly1d([0.317343,1])
# Look at the critical range near kT~mc^2.
rr = np.logspace(-0.75,1.25,200)
ex = np.array([exactJ(r) for r in rr])
pa = num(rr)/den(rr)
#
err=np.abs(ex-pa)/ex
print("Maximum error is {:8.4f}% at {:8.4f}".format(100*np.max(err),rr[np.argmax(err)]))
# Those who recall <p>=3.15 T for a FD particle may not be surprised at where the
# error is maximal.
# Now plot it.
fig,ax = plt.subplots(1,1,figsize=(8,4))
ax.plot(rr,ex,label='Exact')
ax.plot(rr,pa,label='Pade')
ax.legend()
ax.set_xscale('log')
ax.set_yscale('linear')
ax.set_xlabel(r'$r$',fontsize=18)
ax.set_ylabel(r'$\mathcal{J}(r)$',fontsize=18)

Is this good enough?  How big an error on $H(z)$ would we make?

The current upper limit on $m_\nu$ is around $0.1\,$eV.  Let's assume a single massive species for now, with maximal mass, and two relativistic species with $N_{\rm eff}=2.046$ kind-of-massless neutrinos.

In [ ]:
# Set up a cosmology with CDM+baryons, 1 massive neutrino and radiation
# consisting of 2 massless neutrinos plus photons, and Lambda.
Tcmb = 2.725 # K, today
Tnu  = (4./11.)**(1./3.) * Tcmb
print("Tnu={:.3f} K".format(Tnu))
hub  = 0.7   # one digit is enough for now.
Omcb = 0.3   # one digit is enough for now.
Omr  = 2.471e-5*(Tcmb/2.725)**4/hub**2*(1+2.046*7./8.*(4./11.)**(4./3.))
#
mnu  = 0.1   # eV
Omnu = mnu/93.14/hub**2
OmL  = (1.0-Omcb-Omnu)
print("Om_cb={:f}, Om_nu={:f}, Om_r={:f}, OmL={:f}".format(Omcb,Omnu,Omr,OmL))
# At what z does the neutrino become non-relativistic?
# Peak of FD is at 3.15 kT
kB = 8.61733e-5 # eV/K
znr= mnu/(3.15*kB*Tnu) - 1
print("z_nr={:.1f}".format(znr))

In [ ]:
# Work out all of the densities at where our approximation is worst.
# Scale all of the densities to rho_crit today.
zz     = znr
rho_cb = Omcb*(1+zz)**3
rho_r  = Omr *(1+zz)**4
rho_L  = OmL
# Now the neutrinos.  It's actually easier to set the density at
# high-z, where we know J(r)->7/8 than at lower z.
Omnu   = 2.471e-5*(Tnu/2.725)**4/hub**2
rr     = mnu/kB/Tnu/(1+zz)
rho_nu = Omnu * (1+zz)**4 * num(rr)/den(rr)
rho_T  = rho_cb + rho_r + rho_L + rho_nu
#
print("Massive neutrino fraction at z={:.1f} is {:12.4e}".format(zz,rho_nu/rho_T))

In [ ]:
# Let's plot this as a function of z.
zz = np.logspace(0.0,3.7,100)
rho_cb = Omcb*(1+zz)**3
rho_r  = Omr *(1+zz)**4
rho_L  = OmL
# Now the neutrinos.
Omnu   = 2.471e-5*(Tnu/2.725)**4/hub**2
rr     = mnu/kB/Tnu/(1+zz)
rho_nu = Omnu * (1+zz)**4 * num(rr)/den(rr)
rho_T  = rho_cb + rho_r + rho_L + rho_nu
#
fig,ax = plt.subplots(1,1)
ax.plot(zz,rho_cb/rho_T,'k:',label=r'$c+b$')
ax.plot(zz,rho_nu/rho_T,'m:',label=r'$\nu$')
ax.plot(zz,rho_r /rho_T,'r:',label=r'rad')
ax.plot(zz,rho_L /rho_T,'b:',label=r'$\Lambda$')
ax.legend()
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_ylim(1e-4,1.3)
ax.set_xlabel('$z$',fontsize=18)
ax.set_ylabel('$\Omega_i$',fontsize=18)

So we'd be making a 3% error on a component which is 1% of the contribution to $H(z)$ at the worst case.  And for most of the rest of the integral we'd be making a smaller error (and we'd be asymptotically accurate).  So this is probably good enough for our purposes.

In [ ]:
# Putting this together, let's compute chi(z) including massive neutrinos.
def chi(z,OmM=0.3,mnu=0.1,hub=0.7):
    num  = np.poly1d([0.0881182,0.277675,0.875])
    den  = np.poly1d([0.317343,1])
    Lhub = 2997.925   # Mpc/h
    kB   = 8.61733e-5 # eV/K
    Tcmb = 2.725 # K, today
    Tnu  = (4./11.)**(1./3.) * Tcmb
    Omr  = 2.471e-5*(Tcmb/2.725)**4/hub**2*(1+2.046*7./8.*(4./11.)**(4./3.))
    Omnu = 2.471e-5*(Tnu /2.725)**4/hub**2  # Not the z=0 value!!!
    Omcb = OmM - Omnu*num(mnu/kB/Tnu)/den(mnu/kB/Tnu)
    OmL  = 1.0-OmM-Omr
    #
    zp     = np.linspace(0.0,z,5000)
    rr     = mnu/kB/Tnu/(1+zp)
    rho_cb = Omcb*(1+zp)**3
    rho_r  = Omr *(1+zp)**4
    rho_nu = Omnu*(1+zp)**4 * num(rr)/den(rr)
    rho_L  = OmL
    Ez     = np.sqrt(rho_cb+rho_r+rho_nu+rho_L)
    chival = np.trapz(1/Ez,x=zp) * Lhub
    return(chival)
    #
print(chi(1.0)," c.f. previous value")

If we wanted to go to higher redshift doing the integral over $z$ the way it's done above isn't very clever.  Rather than do something intelligent, like switch integration methods, I'll just hammer the Trapezoidal rule but in $\ln(1+z)$.

In [ ]:
def chi(z,OmM=0.3,mnu=0.1,hub=0.7):
    """Uses a ln(1+z) integration instead."""
    num  = np.poly1d([0.0881182,0.277675,0.875])
    den  = np.poly1d([0.317343,1])
    Lhub = 2997.925   # Mpc/h
    kB   = 8.61733e-5 # eV/K
    Tcmb = 2.725 # K, today
    Tnu  = (4./11.)**(1./3.) * Tcmb
    Omr  = 2.471e-5*(Tcmb/2.725)**4/hub**2*(1+2.046*7./8.*(4./11.)**(4./3.))
    Omnu = 2.471e-5*(Tnu /2.725)**4/hub**2  # Not the z=0 value!!!
    Omcb = OmM - Omnu*num(mnu/kB/Tnu)/den(mnu/kB/Tnu)
    OmL  = 1.0-OmM-Omr
    #
    lnzp1  = np.linspace(0.0,np.log(1+z),10000)
    zp1    = np.exp(lnzp1)
    rr     = mnu/kB/Tnu/zp1
    rho_cb = Omcb*(zp1)**3
    rho_r  = Omr *(zp1)**4
    rho_nu = Omnu*(zp1)**4 * num(rr)/den(rr)
    rho_L  = OmL
    Ez     = np.sqrt(rho_cb+rho_r+rho_nu+rho_L)
    chival = np.trapz(zp1/Ez,x=lnzp1) * Lhub
    return(chival)
    #
print(chi(1.0)," c.f. previous value")

In [ ]:
# Now plot the ratio of massive neutrinos to no-massive-neutrinos (fiducial) vs. redshift.
zp1  = np.logspace(0.1,4.0,100)
nchi = np.array([chi(z,0.3,0.1,0.7) for z in zp1-1])
fchi = np.array([chi(z,0.3,0.0,0.7) for z in zp1-1])
#
fig,ax = plt.subplots(1,1,figsize=(8,4.5))
ax.plot(zp1,100*(nchi/fchi-1),'b-')
ax.set_xscale('log')
ax.set_xlabel(r'$1+z$',fontsize=18)
ax.set_ylabel(r'Frac.Diff. (%)',fontsize=18)

So this difference is really tiny -- almost negligible.  However it does depend upon what we've decided to hold fixed.  In the above I assumed that $\omega_{c+b+\nu}$ was specified (today).  In reality we're probably measuring the $\omega_i$ from the shapes of the CMB peaks.  Those are set by the dynamics at $z>10^3$, in which case we should really be holding fixed $\omega_{c+b}$ (since the $\nu$ are relativistic at $z\sim 10^3$).  What does this do?

In [ ]:
def chi(z,Omcb=0.3,mnu=0.1,hub=0.7):
    """Uses a ln(1+z) integration instead."""
    num  = np.poly1d([0.0881182,0.277675,0.875])
    den  = np.poly1d([0.317343,1])
    Lhub = 2997.925   # Mpc/h
    kB   = 8.61733e-5 # eV/K
    Tcmb = 2.725 # K, today
    Tnu  = (4./11.)**(1./3.) * Tcmb
    Omr  = 2.471e-5*(Tcmb/2.725)**4/hub**2*(1+2.046*7./8.*(4./11.)**(4./3.))
    Omnu = 2.471e-5*(Tnu /2.725)**4/hub**2  # Not the z=0 value!!!
    OmM  = Omcb + Omnu*num(mnu/kB/Tnu)/den(mnu/kB/Tnu)
    OmL  = 1.0-OmM-Omr
    #
    lnzp1  = np.linspace(0.0,np.log(1+z),10000)
    zp1    = np.exp(lnzp1)
    rr     = mnu/kB/Tnu/zp1
    rho_cb = Omcb*(zp1)**3
    rho_r  = Omr *(zp1)**4
    rho_nu = Omnu*(zp1)**4 * num(rr)/den(rr)
    rho_L  = OmL
    Ez     = np.sqrt(rho_cb+rho_r+rho_nu+rho_L)
    chival = np.trapz(zp1/Ez,x=lnzp1) * Lhub
    return(chival)
    #
#
# Now plot the ratio of massive neutrinos to no-massive-neutrinos (fiducial) vs. redshift.
zp1  = np.logspace(0.1,4.0,100)
nchi = np.array([chi(z,0.3,0.1,0.7) for z in zp1-1])
fchi = np.array([chi(z,0.3,0.0,0.7) for z in zp1-1])
#
fig,ax = plt.subplots(1,1,figsize=(8,5))
ax.plot(zp1,100*(nchi/fchi-1),'b-')
ax.set_xscale('log')
ax.set_xlabel(r'$1+z$',fontsize=18)
ax.set_ylabel(r'Frac.Diff. (%)',fontsize=18)

This isn't a massive effect, but it's significantly larger than holding $\omega_{c+b+\nu}$ fixed!  Since the distance to last scattering is **so** well measured, this starts to have an effect!